In [ ]:
from TauLidarCommon.frame import FrameType, Frame
from TauLidarCamera.camera import Camera
from TauLidarCamera.constants import VALUE_20MHZ
from TauLidarCommon.color import ColorMode

In [ ]:
camera = Camera.open()

In [ ]:
cameraInfo = camera.info()

print("\nToF camera opened successfully:")
print("    model:      %s" % cameraInfo.model)
print("    firmware:   %s" % cameraInfo.firmware)
print("    uid:        %s" % cameraInfo.uid)
print("    resolution: %s" % cameraInfo.resolution)
print("    port:       %s" % cameraInfo.port)

In [ ]:
camera.setModulationChannel(0) ## autoChannelEnabled: 0, channel: 0

In [ ]:
camera.setIntegrationTime3d(0, 800)  ## set integration time 0: 1000
camera.setMinimalAmplitude(0, 60)  ## set minimal amplitude 0: 80
Camera.setRange(0, 4500) ## points in the distance range to be colored
camera.setIntegrationTimeGrayscale(15000)  ## set integration time grayscale: 8000, needed when requiring FrameType.DISTANCE_GRAYSCALE

In [ ]:
frame = camera.readFrame(FrameType.DISTANCE_AMPLITUDE)

In [ ]:
import datetime
import os
import time
import cv2
import numpy as np

# Start time
start_time = time.time()
print(f"Start Time: {start_time}")
end_time = start_time + 4  # Capture frames for 4 seconds
print(f"End Time: {end_time}")

last_captured_image = None  # To store the last captured frame

while time.time() < end_time:
    # Capture frame
    frame = camera.readFrame(FrameType.DISTANCE_AMPLITUDE)
    mat_amplitude = np.frombuffer(frame.data_amplitude, dtype=np.float32, count=-1, offset=0).reshape(frame.height, frame.width)

    # Convert to 8-bit for display
    cv2.normalize(mat_amplitude, mat_amplitude, 0, 255, cv2.NORM_MINMAX)
    mat_amplitude = mat_amplitude.astype(np.uint8)

    # Apply a colormap to the image
    colored_image = cv2.applyColorMap(mat_amplitude, cv2.COLORMAP_JET)

    # Enlarge the image while preserving aspect ratio
    target_width = 2000
    target_height = 1500
    resized_image = cv2.resize(colored_image, (target_width, target_height), interpolation=cv2.INTER_LINEAR)

    # Display the live feed
    cv2.imshow('Object Detection on Depth Map', resized_image)

    # Store the last captured image
    last_captured_image = resized_image

    # Break if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Close OpenCV window
cv2.destroyAllWindows()

# Define save directory
save_dir = "saved_images_4"
os.makedirs(save_dir, exist_ok=True)

# Generate filename with timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_path = os.path.join(save_dir, f"depth_map_{timestamp}.png")

# Save the last captured image
if last_captured_image is not None:
    cv2.imwrite(file_path, last_captured_image)
    print(f"Image saved successfully at: {file_path}")
else:
    print("No image was captured.")
